In [1]:
from paddleocr import PaddleOCR, draw_ocr
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import io
import PIL
import cv2

import sys
sys.path.append("../../") 

from project.ComputerVision.ocr import OCR

In [2]:
paddleocr = PaddleOCR(show_log=False, use_angle_cls=True, enable_mkldnn=True, lang="latin")

In [28]:
image_path = "../../data/images/menu3.jpg"
im = PIL.Image.open(image_path) 

In [ ]:
def extract_text_from_image(image_path):
    result = paddleocr.ocr(image_path, cls=True)

    ocr_res = []

    for ps, (text, score) in result[0]:
        x1 = min(p[0] for p in ps)
        y1 = min(p[1] for p in ps)
        x2 = max(p[0] for p in ps)
        y2 = max(p[1] for p in ps)
        word_info = {
            'bbox': [x1, y1, x2, y2],
            'text': text
        }
        ocr_res.append(word_info)
    return ocr_res

def find_intersecting_bboxes(listbox, listocr):
    result = []

    for bbox1 in listbox:
        bbox1_x, bbox1_y, bbox1_w, bbox1_h = bbox1
        e = {'bbox_from_listbox' : [bbox1_x, bbox1_y, bbox1_w, bbox1_h], 'bbox_from_listocr':[], 'text_from_listocr':[]}

        for bbox2 in listocr:
            bbox2_x1, bbox2_y1, bbox2_x2, bbox2_y2 = bbox2["bbox"]
            bbox2_center_x = bbox2_x1 + ((bbox2_x2 - bbox2_x1) // 2)
            bbox2_center_y = bbox2_y1 + ((bbox2_y2 - bbox2_y1) // 2)

            if (bbox1_x <= bbox2_center_x <= bbox1_x + bbox1_w) and (bbox1_y <= bbox2_center_y <= bbox1_y + bbox1_h):
                e["bbox_from_listocr"].append([bbox2_x1, bbox2_y1, bbox2_x2, bbox2_y2])
                e["text_from_listocr"].append(bbox2["text"])
        result.append(e)

    return result


def aggregate_bounding_boxes(bounding_boxes):
    if not bounding_boxes:
        return None

    min_x = min(box[0] for box in bounding_boxes)
    min_y = min(box[1] for box in bounding_boxes)
    max_x = max(box[2] for box in bounding_boxes)
    max_y = max(box[3] for box in bounding_boxes)

    aggregated_box = [min_x, min_y, max_x, max_y]
    return aggregated_box

In [17]:
ocr = OCR()

In [29]:
ocr_result = extract_text_from_page(image_path)
box_recipe = ocr.return_cleaned_boxes(image_path)["Recipe"]
box_ocr = [d["bbox"] for d in ocr_result]
res = find_intersecting_bboxes(box_recipe, ocr_result)


image 1/1 C:\Users\EliottCOLSON\OneDrive - Ekimetrics\Documents\Carbon-Diet\carbondiet\data\images\menu3.jpg: 800x576 16 Recipes, 1351.7ms
Speed: 5.2ms preprocess, 1351.7ms inference, 1.0ms postprocess per image at shape (1, 3, 800, 800)


[{'bbox_from_listbox': [355, 514, 277, 56],
  'bbox_from_listocr': [[462.0, 520.0, 528.0, 540.0],
   [366.0, 543.0, 624.0, 562.0]],
  'text_from_listocr': ['MERLU', 'ASPERGES BLANCHES/SAUCEVIERGE']},
 {'bbox_from_listbox': [333, 581, 316, 56],
  'bbox_from_listocr': [[454.0, 584.0, 535.0, 612.0],
   [350.0, 612.0, 640.0, 631.0]],
  'text_from_listocr': ['SÉBASTE', 'RIZGRILLE/ARTICHAUTS/BEURREBLANC']},
 {'bbox_from_listbox': [339, 326, 310, 56],
  'bbox_from_listocr': [[453.0, 332.0, 537.0, 352.0],
   [352.0, 355.0, 640.0, 374.0]],
  'text_from_listocr': ['TOMATES', "MOZZARELLA/TERREAU D'OLIVES NOIRES"]},
 {'bbox_from_listbox': [293, 653, 409, 54],
  'bbox_from_listocr': [[458.0, 658.0, 534.0, 679.0],
   [297.0, 680.0, 693.0, 699.0]],
  'text_from_listocr': ['COCHON',
   'LONGE IBERIQUE/SUCRINE/POMMES DETERRE FUMEES']},
 {'bbox_from_listbox': [339, 396, 314, 50],
  'bbox_from_listocr': [[456.0, 397.0, 533.0, 423.0],
   [346.0, 425.0, 645.0, 444.0]],
  'text_from_listocr': ['RISOTTO', "B

In [32]:
old_bbox = [e["bbox_from_listbox"] for e in res]
new_bbox = [aggregate_bounding_boxes(e["bbox_from_listocr"]) for e in res]
text = [", ".join(e["text_from_listocr"]) for e in res]

In [35]:
boxes = ocr.return_cleaned_boxes(image_path)
ocr.attribute_boxes(boxes)

['MERLU, ASPERGES BLANCHES/SAUCEVIERGE',
 'SÉBASTE, RIZGRILLE/ARTICHAUTS/BEURREBLANC',
 "TOMATES, MOZZARELLA/TERREAU D'OLIVES NOIRES",
 'COCHON, LONGE IBERIQUE/SUCRINE/POMMES DETERRE FUMEES',
 "RISOTTO, BISQUE D'ECREVISSES/TOMATES CERISES",
 'SAUMON, GRAVLAX/BETTERAVES/AVOCAT',
 'BCEUF, TARTARE AU COUTEAU/POMMES PAILLES',
 "BCEUF, ANGUS/CAVIAR D'AUBERGINES/CONDIMENTPOIVRE",
 'CANETTE, FILET/POMMESANNA/LEGUMESDE SAISON',
 'PANNA COTTA, RHUBARBE/ROSE',
 'La Gri!tte',
 'ABRICOT, TARTE/AMANDE/MIEL',
 'CHOCOLAT, MOUSSE/CONDIMENTS',
 'FROMAGES8€, ASSORTIMENT DE TROIS PIECES',
 'ENTREE+PLAT+DESSERT=28€']